In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

C:\Users\ladwi\anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generating the training data for the Heat and Diffusion Model

In [2]:
data_dir = "./LakePIAB/"
depth_steps = 25 * 2

print(os.getcwd())

C:\Users\ladwi\Documents\Projects\R\LakePIAB\MCL\01_data_processing


In [3]:
meterological_data_df = pd.read_csv("./../../output/py_meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg,density_snow,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.816044,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.816044,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.816044,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.816044,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.816044,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.304264,250.0,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.304264,250.0,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.304264,250.0,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.304264,250.0,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481


In [4]:
input_temp_df = pd.read_csv("./../../output/py_temp_total05.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total05':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total05,depth
0,2013-12-30 01:00:00,4.289738,4.089224,1
1,2013-12-30 01:00:00,4.215537,4.203297,2
2,2013-12-30 01:00:00,4.350104,4.309042,3
3,2013-12-30 01:00:00,4.464834,4.426512,4
4,2013-12-30 01:00:00,4.498299,4.493034,5
...,...,...,...,...
1751945,2017-12-28 23:00:00,4.209767,4.209855,46
1751946,2017-12-28 23:00:00,4.303953,4.304061,47
1751947,2017-12-28 23:00:00,4.398347,4.398476,48
1751948,2017-12-28 23:00:00,4.492609,4.492759,49


In [5]:
depth_list.shape

(1751950,)

In [6]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.289738,4.089224
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.215537,4.203297
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.350104,4.309042
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.464834,4.426512
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.498299,4.493034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.209767,4.209855
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.303953,4.304061
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.398347,4.398476
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.287191,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.492609,4.492759


In [7]:
buoyancy_data_df = pd.read_csv("./../../output/py_buoyancy.csv")
#Removing Nans
#buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
#buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']
buoyancy_data_df['24'] = buoyancy_data_df['23']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2013-12-30 01:00:00,0.000007,1
1,2013-12-30 01:00:00,0.000009,2
2,2013-12-30 01:00:00,0.000014,3
3,2013-12-30 01:00:00,0.000010,4
4,2013-12-30 01:00:00,0.000001,5
...,...,...,...
1751945,2017-12-28 23:00:00,0.000008,46
1751946,2017-12-28 23:00:00,0.000011,47
1751947,2017-12-28 23:00:00,0.000014,48
1751948,2017-12-28 23:00:00,0.000016,49


In [8]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.289738,4.089224,0.000007
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.215537,4.203297,0.000009
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.350104,4.309042,0.000014
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.464834,4.426512,0.000010
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.910924,4.498299,4.493034,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.209767,4.209855,0.000008
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.303953,4.304061,0.000011
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.398347,4.398476,0.000014
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.025375,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.492609,4.492759,0.000016


In [9]:
out_diffusivity_df = pd.read_csv("./../../output/py_diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2013-12-30 01:00:00,0.019877,1
1,2013-12-30 01:00:00,0.019877,2
2,2013-12-30 01:00:00,0.000703,3
3,2013-12-30 01:00:00,0.000040,4
4,2013-12-30 01:00:00,0.000008,5
...,...,...,...
1751945,2017-12-28 23:00:00,0.000008,46
1751946,2017-12-28 23:00:00,0.000008,47
1751947,2017-12-28 23:00:00,0.000008,48
1751948,2017-12-28 23:00:00,0.000008,49


In [10]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.910924,4.289738,4.089224,0.000007,0.019877
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.910924,4.215537,4.203297,0.000009,0.019877
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.910924,4.350104,4.309042,0.000014,0.000703
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.910924,4.464834,4.426512,0.000010,0.000040
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.910924,4.498299,4.493034,0.000001,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.209767,4.209855,0.000008,0.000008
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.303953,4.304061,0.000011,0.000008
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.398347,4.398476,0.000014,0.000008
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.044332,250.0,0.746482,1.0,0.8,-0.309481,4.492609,4.492759,0.000016,0.000008


In [11]:
out_temp_df = pd.read_csv("./../../output/py_temp_heat01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_heat01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_heat01,depth
0,2013-12-30 01:00:00,4.089224,1
1,2013-12-30 01:00:00,4.215545,2
2,2013-12-30 01:00:00,4.350109,3
3,2013-12-30 01:00:00,4.464839,4
4,2013-12-30 01:00:00,4.498304,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.209831,46
1751946,2017-12-28 23:00:00,4.304033,47
1751947,2017-12-28 23:00:00,4.398499,48
1751948,2017-12-28 23:00:00,4.492695,49


In [12]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.910924,4.289738,4.089224,0.000007,0.019877,4.089224
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.910924,4.215537,4.203297,0.000009,0.019877,4.215545
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.910924,4.350104,4.309042,0.000014,0.000703,4.350109
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.910924,4.464834,4.426512,0.000010,0.000040,4.464839
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.910924,4.498299,4.493034,0.000001,0.000008,4.498304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,250.0,0.746482,1.0,0.8,-0.309481,4.209767,4.209855,0.000008,0.000008,4.209831
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,250.0,0.746482,1.0,0.8,-0.309481,4.303953,4.304061,0.000011,0.000008,4.304033
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,250.0,0.746482,1.0,0.8,-0.309481,4.398347,4.398476,0.000014,0.000008,4.398499
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,250.0,0.746482,1.0,0.8,-0.309481,4.492609,4.492759,0.000016,0.000008,4.492695


In [13]:
out_temp_df = pd.read_csv("./../../output/py_temp_diff02.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff02':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff02,depth
0,2013-12-30 01:00:00,4.089224,1
1,2013-12-30 01:00:00,4.203297,2
2,2013-12-30 01:00:00,4.309042,3
3,2013-12-30 01:00:00,4.426512,4
4,2013-12-30 01:00:00,4.493034,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.209855,46
1751946,2017-12-28 23:00:00,4.304061,47
1751947,2017-12-28 23:00:00,4.398476,48
1751948,2017-12-28 23:00:00,4.492759,49


In [14]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.910924,4.289738,4.089224,0.000007,0.019877,4.089224,4.089224
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.910924,4.215537,4.203297,0.000009,0.019877,4.215545,4.203297
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.910924,4.350104,4.309042,0.000014,0.000703,4.350109,4.309042
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.910924,4.464834,4.426512,0.000010,0.000040,4.464839,4.426512
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.910924,4.498299,4.493034,0.000001,0.000008,4.498304,4.493034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.746482,1.0,0.8,-0.309481,4.209767,4.209855,0.000008,0.000008,4.209831,4.209855
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.746482,1.0,0.8,-0.309481,4.303953,4.304061,0.000011,0.000008,4.304033,4.304061
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.746482,1.0,0.8,-0.309481,4.398347,4.398476,0.000014,0.000008,4.398499,4.398476
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.746482,1.0,0.8,-0.309481,4.492609,4.492759,0.000016,0.000008,4.492695,4.492759


In [15]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [16]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [17]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.8,5.910924,4.289738,4.089224,0.000007,0.019877,4.089224,4.089224,364,1
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.8,5.910924,4.215537,4.203297,0.000009,0.019877,4.215545,4.203297,364,1
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.8,5.910924,4.350104,4.309042,0.000014,0.000703,4.350109,4.309042,364,1
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.8,5.910924,4.464834,4.426512,0.000010,0.000040,4.464839,4.426512,364,1
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.8,5.910924,4.498299,4.493034,0.000001,0.000008,4.498304,4.493034,364,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.8,-0.309481,4.209767,4.209855,0.000008,0.000008,4.209831,4.209855,362,23
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.8,-0.309481,4.303953,4.304061,0.000011,0.000008,4.304033,4.304061,362,23
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.8,-0.309481,4.398347,4.398476,0.000014,0.000008,4.398499,4.398476,362,23
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.8,-0.309481,4.492609,4.492759,0.000016,0.000008,4.492695,4.492759,362,23


In [18]:
temp_mix03_df = pd.read_csv("./../../output/py_temp_mix03.csv")

temp_mix03_df = temp_mix03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix03':flattened_temp, 'depth':depth_list}
temp_mix03_df = pd.DataFrame(data=data)

temp_mix03_df

,time,temp_mix03,depth
0,2013-12-30 01:00:00,4.089224,1
1,2013-12-30 01:00:00,4.203297,2
2,2013-12-30 01:00:00,4.309042,3
3,2013-12-30 01:00:00,4.426512,4
4,2013-12-30 01:00:00,4.493034,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.209855,46
1751946,2017-12-28 23:00:00,4.304061,47
1751947,2017-12-28 23:00:00,4.398476,48
1751948,2017-12-28 23:00:00,4.492759,49


In [19]:
final_df = final_df.merge(temp_mix03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.910924,4.289738,4.089224,0.000007,0.019877,4.089224,4.089224,364,1,4.089224
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.910924,4.215537,4.203297,0.000009,0.019877,4.215545,4.203297,364,1,4.203297
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.910924,4.350104,4.309042,0.000014,0.000703,4.350109,4.309042,364,1,4.309042
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.910924,4.464834,4.426512,0.000010,0.000040,4.464839,4.426512,364,1,4.426512
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,5.910924,4.498299,4.493034,0.000001,0.000008,4.498304,4.493034,364,1,4.493034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.309481,4.209767,4.209855,0.000008,0.000008,4.209831,4.209855,362,23,4.209855
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.309481,4.303953,4.304061,0.000011,0.000008,4.304033,4.304061,362,23,4.304061
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.309481,4.398347,4.398476,0.000014,0.000008,4.398499,4.398476,362,23,4.398476
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,-0.309481,4.492609,4.492759,0.000016,0.000008,4.492695,4.492759,362,23,4.492759


In [20]:
temp_conv03_df = pd.read_csv("./../../output/py_temp_conv04.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv04':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv04,depth
0,2013-12-30 01:00:00,4.089224,1
1,2013-12-30 01:00:00,4.203297,2
2,2013-12-30 01:00:00,4.309042,3
3,2013-12-30 01:00:00,4.426512,4
4,2013-12-30 01:00:00,4.493034,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.209855,46
1751946,2017-12-28 23:00:00,4.304061,47
1751947,2017-12-28 23:00:00,4.398476,48
1751948,2017-12-28 23:00:00,4.492759,49


In [21]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.289738,4.089224,0.000007,0.019877,4.089224,4.089224,364,1,4.089224,4.089224
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.215537,4.203297,0.000009,0.019877,4.215545,4.203297,364,1,4.203297,4.203297
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.350104,4.309042,0.000014,0.000703,4.350109,4.309042,364,1,4.309042,4.309042
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.464834,4.426512,0.000010,0.000040,4.464839,4.426512,364,1,4.426512,4.426512
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.498299,4.493034,0.000001,0.000008,4.498304,4.493034,364,1,4.493034,4.493034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.209767,4.209855,0.000008,0.000008,4.209831,4.209855,362,23,4.209855,4.209855
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.303953,4.304061,0.000011,0.000008,4.304033,4.304061,362,23,4.304061,4.304061
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.398347,4.398476,0.000014,0.000008,4.398499,4.398476,362,23,4.398476,4.398476
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.492609,4.492759,0.000016,0.000008,4.492695,4.492759,362,23,4.492759,4.492759


In [22]:
temp_initial00_df = pd.read_csv("./../../output/py_temp_initial00.csv")

temp_initial00_df = temp_initial00_df[1:] # considering everything from 2nd time step

flattened_temp = temp_initial00_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_initial00_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_initial00':flattened_temp, 'depth':depth_list}
temp_initial00_df = pd.DataFrame(data=data)

temp_initial00_df

,time,temp_initial00,depth
0,2013-12-30 01:00:00,4.289738,1
1,2013-12-30 01:00:00,4.215537,2
2,2013-12-30 01:00:00,4.350104,3
3,2013-12-30 01:00:00,4.464834,4
4,2013-12-30 01:00:00,4.498299,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.209767,46
1751946,2017-12-28 23:00:00,4.303953,47
1751947,2017-12-28 23:00:00,4.398347,48
1751948,2017-12-28 23:00:00,4.492609,49


In [23]:
final_df = final_df.merge(temp_initial00_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.089224,0.000007,0.019877,4.089224,4.089224,364,1,4.089224,4.089224,4.289738
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.203297,0.000009,0.019877,4.215545,4.203297,364,1,4.203297,4.203297,4.215537
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.309042,0.000014,0.000703,4.350109,4.309042,364,1,4.309042,4.309042,4.350104
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.426512,0.000010,0.000040,4.464839,4.426512,364,1,4.426512,4.426512,4.464834
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,4.493034,0.000001,0.000008,4.498304,4.493034,364,1,4.493034,4.493034,4.498299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.209855,0.000008,0.000008,4.209831,4.209855,362,23,4.209855,4.209855,4.209767
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.304061,0.000011,0.000008,4.304033,4.304061,362,23,4.304061,4.304061,4.303953
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.398476,0.000014,0.000008,4.398499,4.398476,362,23,4.398476,4.398476,4.398347
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,4.492759,0.000016,0.000008,4.492695,4.492759,362,23,4.492759,4.492759,4.492609


In [24]:
temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}

print(data)
print(len(data['time']))
print(len(data['obs_temp']))
print(len(data['depth']))

temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

{'time': array(['2013-12-30 01:00:00', '2013-12-30 01:00:00',
       '2013-12-30 01:00:00', ..., '2017-12-28 23:00:00',
       '2017-12-28 23:00:00', '2017-12-28 23:00:00'], dtype=object), 'obs_temp': array([2.5327381 , 2.70760582, 2.70760582, ..., 4.00578453, 4.82646421,
       7.99623398]), 'depth': array([ 1,  2,  3, ..., 48, 49, 50])}
1751950
1751950
1751950
(1751950,)
(1751950,)
(1751950,)


In [25]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000007,0.019877,4.089224,4.089224,364,1,4.089224,4.089224,4.289738,2.532738
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000009,0.019877,4.215545,4.203297,364,1,4.203297,4.203297,4.215537,2.707606
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000014,0.000703,4.350109,4.309042,364,1,4.309042,4.309042,4.350104,2.707606
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000010,0.000040,4.464839,4.426512,364,1,4.426512,4.426512,4.464834,2.707606
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000001,0.000008,4.498304,4.493034,364,1,4.493034,4.493034,4.498299,2.707606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000008,0.000008,4.209831,4.209855,362,23,4.209855,4.209855,4.209767,3.029762
1751946,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000011,0.000008,4.304033,4.304061,362,23,4.304061,4.304061,4.303953,3.185105
1751947,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000014,0.000008,4.398499,4.398476,362,23,4.398476,4.398476,4.398347,4.005785
1751948,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000016,0.000008,4.492695,4.492759,362,23,4.492759,4.492759,4.492609,4.826464


In [26]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)


time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(1751950,)
(1751950,)
(1751950,)


,time,input_obs,depth
0,2013-12-30 01:00:00,2.535714,1
1,2013-12-30 01:00:00,2.710317,2
2,2013-12-30 01:00:00,2.710317,3
3,2013-12-30 01:00:00,2.710317,4
4,2013-12-30 01:00:00,2.710317,5
...,...,...,...
1751945,2017-12-28 23:00:00,3.030952,46
1751946,2017-12-28 23:00:00,3.189226,47
1751947,2017-12-28 23:00:00,4.009906,48
1751948,2017-12-28 23:00:00,4.830586,49


In [27]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.019877,4.089224,4.089224,364,1,4.089224,4.089224,4.289738,2.532738,2.535714
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.019877,4.215545,4.203297,364,1,4.203297,4.203297,4.215537,2.707606,2.710317
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000703,4.350109,4.309042,364,1,4.309042,4.309042,4.350104,2.707606,2.710317
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000040,4.464839,4.426512,364,1,4.426512,4.426512,4.464834,2.707606,2.710317
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,0.000008,4.498304,4.493034,364,1,4.493034,4.493034,4.498299,2.707606,2.710317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752345,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000008,4.209831,4.209855,362,23,4.209855,4.209855,4.209767,3.029762,3.030952
1752346,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000008,4.304033,4.304061,362,23,4.304061,4.304061,4.303953,3.185105,3.189226
1752347,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000008,4.398499,4.398476,362,23,4.398476,4.398476,4.398347,4.005785,4.009906
1752348,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,0.000008,4.492695,4.492759,362,23,4.492759,4.492759,4.492609,4.826464,4.830586


In [28]:
ice_data_df = pd.read_csv("./../../output/py_icesnow.csv")
ice_data_df = ice_data_df[1:] # considering everything from 2nd time step

num_time_steps = ice_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
ice_data_df = pd.DataFrame(np.repeat(ice_data_df.values, depth_steps, axis=0), columns=ice_data_df.columns)
ice_data_df = pd.concat([depth_df, ice_data_df], ignore_index=False, axis=1)
print(ice_data_df)

final_df = final_df.merge(ice_data_df, how='inner', on=['time','depth'])
final_df

         depth                 time       ice      snow   snowice
0            1  2013-12-30 01:00:00       0.0       0.0       0.0
1            2  2013-12-30 01:00:00       0.0       0.0       0.0
2            3  2013-12-30 01:00:00       0.0       0.0       0.0
3            4  2013-12-30 01:00:00       0.0       0.0       0.0
4            5  2013-12-30 01:00:00       0.0       0.0       0.0
...        ...                  ...       ...       ...       ...
1751945     46  2017-12-28 23:00:00  0.287239  0.025556  0.044332
1751946     47  2017-12-28 23:00:00  0.287239  0.025556  0.044332
1751947     48  2017-12-28 23:00:00  0.287239  0.025556  0.044332
1751948     49  2017-12-28 23:00:00  0.287239  0.025556  0.044332
1751949     50  2017-12-28 23:00:00  0.287239  0.025556  0.044332

[1751950 rows x 5 columns]


,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs,ice,snow,snowice
0,1,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,364,1,4.089224,4.089224,4.289738,2.532738,2.535714,0.0,0.0,0.0
1,2,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,364,1,4.203297,4.203297,4.215537,2.707606,2.710317,0.0,0.0,0.0
2,3,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,364,1,4.309042,4.309042,4.350104,2.707606,2.710317,0.0,0.0,0.0
3,4,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,364,1,4.426512,4.426512,4.464834,2.707606,2.710317,0.0,0.0,0.0
4,5,2013-12-30 01:00:00,-1.252423,584.398073,-21.60148,-26.402849,0.0,0.4,-999.0,-999.0,...,364,1,4.493034,4.493034,4.498299,2.707606,2.710317,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752345,46,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,362,23,4.209855,4.209855,4.209767,3.029762,3.030952,0.287239,0.025556,0.044332
1752346,47,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,362,23,4.304061,4.304061,4.303953,3.185105,3.189226,0.287239,0.025556,0.044332
1752347,48,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,362,23,4.398476,4.398476,4.398347,4.005785,4.009906,0.287239,0.025556,0.044332
1752348,49,2017-12-28 23:00:00,-3.068643,528.790329,-15.910644,-20.374966,0.0,0.4,-999.0,-999.0,...,362,23,4.492759,4.492759,4.492609,4.826464,4.830586,0.287239,0.025556,0.044332


In [29]:
obs_array = final_df['obs_temp']
obs_array[obs_array == -999] = final_df['temp_total05']
print(obs_array)
final_df['obs_temp'] = obs_array

0          2.532738
1          2.707606
2          2.707606
3          2.707606
4          2.707606
             ...   
1752345    3.029762
1752346    3.185105
1752347    4.005785
1752348    4.826464
1752349    7.996234
Name: obs_temp, Length: 1752350, dtype: float64


C:\Users\ladwi\AppData\Local\Temp\ipykernel_20684\1668945599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_array[obs_array == -999] = final_df['temp_total05']


In [30]:
final_df_null = final_df.fillna('')
print(final_df_null.head)

<bound method NDFrame.head of          depth                 time  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  \
0            1  2013-12-30 01:00:00     -1.252423     584.398073   -21.601480   
1            2  2013-12-30 01:00:00     -1.252423     584.398073   -21.601480   
2            3  2013-12-30 01:00:00     -1.252423     584.398073   -21.601480   
3            4  2013-12-30 01:00:00     -1.252423     584.398073   -21.601480   
4            5  2013-12-30 01:00:00     -1.252423     584.398073   -21.601480   
...        ...                  ...           ...            ...          ...   
1752345     46  2017-12-28 23:00:00     -3.068643     528.790329   -15.910644   
1752346     47  2017-12-28 23:00:00     -3.068643     528.790329   -15.910644   
1752347     48  2017-12-28 23:00:00     -3.068643     528.790329   -15.910644   
1752348     49  2017-12-28 23:00:00     -3.068643     528.790329   -15.910644   
1752349     50  2017-12-28 23:00:00     -3.068643     528.790329   -15.910644  

In [31]:
final_df_null.to_csv("./../02_training/all_data_lake_modeling_in_time.csv", index=False)

In [32]:
final_df_null.to_csv("all_data_lake_modeling_in_time.csv", index=False)